# Seminar 2: SLY(Sly Lex Yacc)
## Lexer analyzer

### Introduction

SLY is library for writing parsers and compilers. It is loosely based on the traditional compiler construction tools lex and yacc and implements the same LALR(1) parsing algorithm. Most of the features available in lex and yacc are also available in SLY.

SLY provides two separate classes Lexer and Parser. The Lexer class is used to break input text into a collection of tokens specified by a collection of regular expression rules. The Parser class is used to recognize language syntax that has been specified in the form of a context free grammar. The two classes are typically used together to make a parser. However, this is not a strict requirement–there is a great deal of flexibility allowed. The next two parts describe the basics.

### Writing a Lexer

Suppose you are writing a programming language and you wanted to parse the following input string:

x = 3 + 42 * (s - t)

The first step of parsing is to break the text into tokens where each token has a type and value. For example, the above text might be described by the following list of token tuples:

[ ('ID','x'), ('EQUALS','='), ('NUMBER','3'),
  ('PLUS','+'), ('NUMBER','42'), ('TIMES','*'),
  ('LPAREN','('), ('ID','s'), ('MINUS','-'),
  ('ID','t'), ('RPAREN',')') ]
  
The SLY Lexer class is used to do this. Here is a sample of a simple lexer that tokenizes the above text:

In [7]:
# calclex.py

from sly import Lexer

class CalcLexer(Lexer):
    #Set of token names.   This is always required
    tokens={ID, NUMBER, PLUS, MINUS, TIMES, DIVIDE, ASSIGN, LPAREN, RPAREN }

    #String containing ignored characters between tokens
    ignore = ' \t'

    #Regular expression rules for tokens
    ID      = r'[a-zA-Z][a-zA-Z0-9]*'
    NUMBER  = r'\d+'
    PLUS    = r'\+'
    MINUS   = r'-'
    TIMES   = r'\*'
    DIVIDE  = r'/'
    ASSIGN  = r'='
    LPAREN  = r'\('
    RPAREN  = r'\)'
    #\ is used to specify the caracter not a RE function.

if __name__ == '__main__':
    data = 'x = 3 + 42 * (s2 - tqw)'
    lexer = CalcLexer()
    for tok in lexer.tokenize(data):
        print('type=%r, value=%r' % (tok.type, tok.value))

type='ID', value='x'
type='ASSIGN', value='='
type='NUMBER', value='3'
type='PLUS', value='+'
type='NUMBER', value='42'
type='TIMES', value='*'
type='LPAREN', value='('
type='ID', value='s2'
type='MINUS', value='-'
type='ID', value='tqw'
type='RPAREN', value=')'


### The tokens set
Lexers must specify a tokens set that defines all of the possible token type names that can be produced by the lexer. This is always required and is used to perform a variety of validation checks.

In the example, the following code specified the token names:

    

In [15]:
class CalcLexer(Lexer):
    ...
    # Set of token names.   This is always required
    tokens = { ID, NUMBER, PLUS, MINUS, TIMES,
               DIVIDE, ASSIGN, LPAREN, RPAREN }
    ...


Token names should be specified using all-caps as shown.

### Specification of token match patterns
Tokens are specified by writing a regular expression rule compatible with the re module. The name of each rule must match one of the names of the tokens provided in the tokens set. For example:

In [9]:
PLUS = r'\+'
MINUS = r'-'

Regular expression patterns are compiled using the re.VERBOSE flag which can be used to help readability. However, unescaped whitespace is ignored and comments are allowed in this mode. If your pattern involves whitespace, make sure you use \s. If you need to match the # character, use [#] or \#.

Tokens are matched in the same order that patterns are listed in the Lexer class. Longer tokens always need to be specified before short tokens. For example, if you wanted to have separate tokens for = and ==, you need to make sure that == is listed first. For example:

**IMPORTANT**, LONGER TO SHORTER 

In [11]:
class MyLexer(Lexer):
    tokens = { ASSIGN, EQ, ...}
    ...
    EQ     = r'=='       # MUST APPEAR FIRST! (LONGER)
    ASSIGN = r'='

### Discarded text
The special ignore specification is reserved for single characters that should be completely ignored between tokens in the input stream. Usually this is used to skip over whitespace and other non-essential characters. The characters given in ignore are not ignored when such characters are part of other regular expression patterns. For example, if you had a rule to capture quoted text, that pattern can include the ignored characters (which will be captured in the normal way). The main purpose of ignore is to ignore whitespace and other padding between the tokens that you actually want to parse.

You can also discard more specialized text patterns by writing special regular expression rules with a name that includes the prefix ignore_. For example, this lexer includes rules to ignore comments and newlines:

In [26]:
# calclex.py

from sly import Lexer

class CalcLexer(Lexer):
    tokens={ID, NUMBER, PLUS, MINUS, TIMES, DIVIDE, ASSIGN, LPAREN, RPAREN }
    # String containing ignored characters (between tokens)
    ignore = ' \t'

    # Other ignored patterns
    ignore_comment = r'\#.*'
    ignore_newline = r'\n+'
    ID      = r'[a-zA-Z][a-zA-Z0-9]*'
    NUMBER  = r'\d+'
    def NUMBER(self, t):
        t.value = int(t.value)
        return t
    PLUS    = r'\+'
    MINUS   = r'-'
    TIMES   = r'\*'
    DIVIDE  = r'/'
    ASSIGN  = r'='
    LPAREN  = r'\('
    RPAREN  = r'\)'

if __name__ == '__main__':
    data = '''x = 3 + 42
                * (s    # This is a comment
                    - t)'''
    lexer = CalcLexer()
    for tok in lexer.tokenize(data):
        print('type=%r, value=%r' % (tok.type, tok.value))

type='ID', value='x'
type='ASSIGN', value='='
type='NUMBER', value=3
type='PLUS', value='+'
type='NUMBER', value=42
type='TIMES', value='*'
type='LPAREN', value='('
type='ID', value='s'
type='MINUS', value='-'
type='ID', value='t'
type='RPAREN', value=')'


### Adding Match Actions
When certain tokens are matched, you may want to trigger some kind of action that performs extra processing. For example, converting a numeric value or looking up language keywords. One way to do this is to write your action as a method and give the associated regular expression using the @_() decorator like this:

In [23]:
@_(r'\d+') #Me resulta personalmente fea.
def NUMBER(self, t):
    t.value = int(t.value)   # Convert to a numeric value
    return t

TypeError: 'str' object is not callable

The method always takes a single argument which is an instance of type Token. By default, t.type is set to the name of the token (e.g., 'NUMBER'). The function can change the token type and value as it sees appropriate. When finished, the resulting token object should be returned as a result. If no value is returned by the function, the token is discarded and the next token read.

The @_() decorator is defined automatically within the Lexer class–you don’t need to do any kind of special import for it. It can also accept multiple regular expression rules. For example:

In [ ]:
@_(r'0x[0-9a-fA-F]+', #Me sigue sin gustar jeje
   r'\d+')
def NUMBER(self, t):
    if t.value.startswith('0x'):
        t.value = int(t.value[2:], 16)
    else:
        t.value = int(t.value)
    return t

Instead of using the @_() decorator, you can also write a method that matches the same name as a token previously specified as a string. For example:

In [ ]:
NUMBER = r'\d+'
...
def NUMBER(self, t):
    t.value = int(t.value)
    return t

This is potentially useful trick for debugging a lexer. You can temporarily attach a method a token and have it execute when the token is encountered. If you later take the method away, the lexer will revert back to its original behavior.

### Token Remapping
Occasionally, you might need to remap tokens based on special cases. Consider the case of matching identifiers such as “abc”, “python”, or “guido”. Certain identifiers such as “if”, “else”, and “while” might need to be treated as special keywords. To handle this, include token remapping rules when writing the lexer like this:

In [27]:
# calclex.py

from sly import Lexer

class CalcLexer(Lexer):
    tokens = { ID, IF, ELSE, WHILE }
    # String containing ignored characters (between tokens)
    ignore = ' \t'

    # Base ID rule
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'

    # Special cases
    ID['if'] = IF
    ID['else'] = ELSE
    ID['while'] = WHILE


When parsing an identifier, the special cases will remap certain matching values to a new token type. For example, if the value of an identifier is “if” above, an IF token will be generated.

### Line numbers and position tracking
By default, lexers know nothing about line numbers. This is because they don’t know anything about what constitutes a “line” of input (e.g., the newline character or even if the input is textual data). To update this information, you need to add a special rule for newlines. Promote the ignore_newline rule to a method like this:

In [ ]:
# Define a rule so we can track line numbers
@_(r'\n+')
def ignore_newline(self, t):
    self.lineno += len(t.value)

Within the rule, the lineno attribute of the lexer is now updated. After the line number is updated, the token is discarded since nothing is returned.

Lexers do not perform and kind of automatic column tracking. However, it does record positional information related to each token in the token’s index attribute. Using this, it is usually possible to compute column information as a separate step. For instance, you can search backwards until you reach the previous newline:

In [ ]:
# Compute column.
#     input is the input text string
#     token is a token instance
def find_column(text, token):
    last_cr = text.rfind('\n', 0, token.index)
    if last_cr < 0:
        last_cr = 0
    column = (token.index - last_cr) + 1
    return column

Since column information is often only useful in the context of error handling, calculating the column position can be performed when needed as opposed to including it on each token.

Literal characters
Literal characters can be specified by defining a set literals in the class. For example:

In [29]:
class MyLexer(Lexer):
    ...
    literals = { '+','-','*','/' }
    ...

LexerBuildError: MyLexer class does not define a tokens attribute

A literal character is a single character that is returned “as is” when encountered by the lexer. Literals are checked after all of the defined regular expression rules. Thus, if a rule starts with one of the literal characters, it will always take precedence.

When a literal token is returned, both its type and value attributes are set to the character itself. For example, '+'.

It’s possible to write token methods that perform additional actions when literals are matched. However, you’ll need to set the token type appropriately. For example:

In [ ]:
class MyLexer(Lexer):

     literals = { '{', '}' }

     def __init__(self):
         self.nesting_level = 0

     @_(r'\{')
     def lbrace(self, t):
         t.type = '{'      # Set token type to the expected literal
         self.nesting_level += 1
         return t

     @_(r'\}')
     def rbrace(t):
         t.type = '}'      # Set token type to the expected literal
         self.nesting_level -=1
         return t

### Error handling
If a bad character is encountered while lexing, tokenizing will stop. However, you can add an error() method to handle lexing errors that occur when illegal characters are detected. The error method receives a Token where the value attribute contains all remaining untokenized text. A typical handler might look at this text and skip ahead in some manner. For example:

In [ ]:
class MyLexer(Lexer):
    ...
    # Error handling rule
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

In this case, we print the offending character and skip ahead one character by updating the lexer position. Error handling in a parser is often a hard problem. An error handler might scan ahead to a logical synchronization point such as a semicolon, a blank line, or similar landmark.

If the error() method also returns the passed token, it will show up as an ERROR token in the resulting token stream. This might be useful if the parser wants to see error tokens for some reason–perhaps for the purposes of improved error messages or some other kind of error handling.

### A More Complete Example
Here is a more complete example that puts many of these concepts into practice:

In [36]:
# calclex.py

from sly import Lexer

class CalcLexer(Lexer):
    # Set of token names.   This is always required
    tokens = { NUMBER, ID, WHILE, IF, ELSE, PRINT,
               PLUS, MINUS, TIMES, DIVIDE, ASSIGN,
               EQ, LT, LE, GT, GE, NE }


    literales = { '(', ')', '{', '}', ';' }

    # String containing ignored characters
    ignore = ' \t'

    # Regular expression rules for tokens
    PLUS    = r'\+'
    MINUS   = r'-'
    TIMES   = r'\*'
    DIVIDE  = r'/'
    EQ      = r'=='
    ASSIGN  = r'='
    LE      = r'<='
    LT      = r'<'
    GE      = r'>='
    GT      = r'>'   
    NE      = r'!='

    @_(r'\d+')
    def NUMBER(self, t):
        t.value = int(t.value)
        return t

    # Identifiers and keywords
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    ID['if'] = IF
    ID['else'] = ELSE
    ID['while'] = WHILE
    ID['print'] = PRINT

    ignore_comment = r'\#.*'

    # Line number tracking
    @_(r'\n+')
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')

    def error(self, t):
        print('Line %d: Bad character %r' % (self.lineno, t.value[0]))
        self.index += 1

if __name__ == '__main__':
    data = '''
# Counting
x = 0;
while (x < 10) {
    print x:
    x = x + 1;
}
'''
    lexer = CalcLexer()
    for tok in lexer.tokenize(data):
        print(tok)

Token(type='ID', value='x', lineno=3, index=12)
Token(type='ASSIGN', value='=', lineno=3, index=14)
Token(type='NUMBER', value=0, lineno=3, index=16)
Line 3: Bad character ';'
Token(type='WHILE', value='while', lineno=4, index=19)
Line 4: Bad character '('
Token(type='ID', value='x', lineno=4, index=26)
Token(type='LT', value='<', lineno=4, index=28)
Token(type='NUMBER', value=10, lineno=4, index=30)
Line 4: Bad character ')'
Line 4: Bad character '{'
Token(type='PRINT', value='print', lineno=5, index=40)
Token(type='ID', value='x', lineno=5, index=46)
Line 5: Bad character ':'
Token(type='ID', value='x', lineno=6, index=53)
Token(type='ASSIGN', value='=', lineno=6, index=55)
Token(type='ID', value='x', lineno=6, index=57)
Token(type='PLUS', value='+', lineno=6, index=59)
Token(type='NUMBER', value=1, lineno=6, index=61)
Line 6: Bad character ';'
Line 7: Bad character '}'


Study this example closely. It might take a bit to digest, but all of the essential parts of writing a lexer are there. Tokens have to be specified with regular expression rules. You can optionally attach actions that execute when certain patterns are encountered. Certain features such as character literals are there mainly for convenience, saving you the trouble of writing separate regular expression rules. You can also add error handling.
